In [1]:
import torch
import torchvision
import numpy as np
from ultralytics import YOLO
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.transforms import functional as F
from tqdm import tqdm

import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
import seaborn as sns
from datetime import datetime

In [2]:
CLASS_NAMES=["background", "defectCell"]

class ConsensusEnsemble:
    def __init__(self, fast_rcnn_path, yolo_path, device='cuda'):
        self.device = torch.device(device)
        self.fast_rcnn = self.load_fast_rcnn(fast_rcnn_path)
        self.yolo = YOLO(yolo_path)
        
        # Parámetros de consenso
        self.consensus_iou_threshold = 0.6
        self.min_confidence = 0.7
        
    def load_fast_rcnn(self, model_path):
        """Carga el modelo Fast-RCNN"""
        weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
        model = fasterrcnn_resnet50_fpn_v2(weights=weights)
        
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, len(CLASS_NAMES))
        
        model.load_state_dict(torch.load(model_path, map_location=self.device))
        model.to(self.device)
        model.eval()
        
        return model
    
    def predict_fast_rcnn(self, image_path, conf_threshold=0.4):
        """Predicción con Fast-RCNN"""
        img = torchvision.io.read_image(image_path)
        img = img.float() / 255.0
        img_tensor = img.unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            predictions = self.fast_rcnn(img_tensor)[0]
        
        keep = predictions['scores'] > conf_threshold
        
        return {
            'boxes': predictions['boxes'][keep].cpu().numpy().tolist(),
            'scores': predictions['scores'][keep].cpu().numpy().tolist(),
            'labels': predictions['labels'][keep].cpu().numpy().tolist()
        }
    
    def predict_yolo(self, image_path, conf_threshold=0.4):
        """Predicción con YOLO"""
        results = self.yolo(image_path, conf=conf_threshold)
        
        if len(results[0].boxes) > 0:
            boxes = results[0].boxes.xyxy.cpu().numpy().tolist()
            scores = results[0].boxes.conf.cpu().numpy().tolist()
            # Convertir etiquetas YOLO a formato Fast-RCNN (YOLO: 0=defectCell -> Fast-RCNN: 1=defectCell)
            labels = (results[0].boxes.cls.cpu().numpy() + 1).tolist()
        else:
            boxes, scores, labels = [], [], []
        
        return {'boxes': boxes, 'scores': scores, 'labels': labels}
    
    def calculate_iou(self, box1, box2):
        """Calcula IoU entre dos cajas"""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        if x2 <= x1 or y2 <= y1:
            return 0.0
        
        intersection = (x2 - x1) * (y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0.0
    
    def find_consensus(self, fast_rcnn_pred, yolo_pred):
        """
        Encuentra detecciones con consenso entre ambos modelos
        Solo mantiene detecciones que ambos modelos detectan
        """
        if not fast_rcnn_pred['boxes'] or not yolo_pred['boxes']:
            return {'boxes': [], 'scores': [], 'labels': []}
        
        consensus_results = {'boxes': [], 'scores': [], 'labels': []}
        
        # Para cada detección de Fast-RCNN, buscar consenso en YOLO
        for i, box1 in enumerate(fast_rcnn_pred['boxes']):
            label1 = fast_rcnn_pred['labels'][i]
            score1 = fast_rcnn_pred['scores'][i]
            
            best_iou = 0
            best_match_idx = -1
            
            # Buscar la mejor coincidencia en YOLO
            for j, box2 in enumerate(yolo_pred['boxes']):
                label2 = yolo_pred['labels'][j]
                
                if label1 == label2:  # Misma clase
                    iou = self.calculate_iou(box1, box2)
                    if iou > best_iou and iou > self.consensus_iou_threshold:
                        best_iou = iou
                        best_match_idx = j
            
            # Si hay consenso suficiente, agregar detección
            if best_match_idx >= 0:
                box2 = yolo_pred['boxes'][best_match_idx]
                score2 = yolo_pred['scores'][best_match_idx]
                
                # Promediar coordenadas y scores
                avg_box = [(box1[k] + box2[k]) / 2 for k in range(4)]
                avg_score = (score1 + score2) / 2
                
                # Solo agregar si la confianza promedio es suficientemente alta
                if avg_score >= self.min_confidence:
                    consensus_results['boxes'].append(avg_box)
                    consensus_results['scores'].append(avg_score)
                    consensus_results['labels'].append(label1)
        
        return consensus_results
    
    def predict(self, image_path):
        """
        Predicción principal usando consenso
        """
        # Obtener predicciones de ambos modelos
        fast_rcnn_pred = self.predict_fast_rcnn(image_path)
        yolo_pred = self.predict_yolo(image_path)
        
        print(f"Fast-RCNN: {len(fast_rcnn_pred['boxes'])} detecciones")
        print(f"YOLO: {len(yolo_pred['boxes'])} detecciones")
        
        # Encontrar consenso
        consensus_result = self.find_consensus(fast_rcnn_pred, yolo_pred)
        
        print(f"Consenso: {len(consensus_result['boxes'])} detecciones")
        
        return consensus_result

In [3]:
class ConsensusEnsembleEvaluator:
    def __init__(self, consensus_ensemble, output_dir="./ensemble_evaluation"):
        self.ensemble = consensus_ensemble
        self.output_dir = output_dir
        self.create_output_dirs()
        
        # Métricas acumuladas
        self.all_predictions = []
        self.all_ground_truth = []
        self.detection_metrics = {
            'true_positives': 0,
            'false_positives': 0, 
            'false_negatives': 0,
            'total_predictions': 0,
            'total_ground_truth': 0
        }
        
    def create_output_dirs(self):
        """Crear directorios de salida"""
        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(os.path.join(self.output_dir, "visualizations"), exist_ok=True)
        os.makedirs(os.path.join(self.output_dir, "metrics"), exist_ok=True)
        os.makedirs(os.path.join(self.output_dir, "results"), exist_ok=True)
        
    def evaluate_test_set(self, test_images_dir, test_labels_dir=None, 
                         iou_threshold=0.5, save_visualizations=True):
        """
        Evalúa todo el conjunto de test
        
        Args:
            test_images_dir: Directorio con imágenes de test
            test_labels_dir: Directorio con labels YOLO (.txt)
            iou_threshold: Umbral IoU para considerar TP
            save_visualizations: Si guardar imágenes con boxes
        """
        print(f"🚀 Iniciando evaluación del conjunto de test...")
        print(f"📁 Directorio de imágenes: {test_images_dir}")
        print(f"📁 Directorio de salida: {self.output_dir}")
        
        # Obtener lista de imágenes
        image_files = [f for f in os.listdir(test_images_dir) 
                      if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        print(f"📊 Total de imágenes a evaluar: {len(image_files)}")
        
        results_data = []
        
        for i, image_file in enumerate(image_files):
            print(f"🔍 Procesando {i+1}/{len(image_files)}: {image_file}")
            
            image_path = os.path.join(test_images_dir, image_file)
            
            # Cargar ground truth si existe
            gt_boxes, gt_labels = self.load_ground_truth(image_file, test_labels_dir)
            
            # Hacer predicción con ensemble
            predictions = self.ensemble.predict(image_path)
            
            # Calcular métricas para esta imagen
            image_metrics = self.calculate_image_metrics(
                predictions, gt_boxes, gt_labels, iou_threshold
            )
            
            # Guardar datos de esta imagen
            result_data = {
                'image_name': image_file,
                'predictions_count': len(predictions['boxes']),
                'ground_truth_count': len(gt_boxes),
                'true_positives': image_metrics['tp'],
                'false_positives': image_metrics['fp'],
                'false_negatives': image_metrics['fn'],
                'precision': image_metrics['precision'],
                'recall': image_metrics['recall'],
                'f1': image_metrics['f1']
            }
            results_data.append(result_data)
            
            # Actualizar métricas globales
            self.detection_metrics['true_positives'] += image_metrics['tp']
            self.detection_metrics['false_positives'] += image_metrics['fp']
            self.detection_metrics['false_negatives'] += image_metrics['fn']
            self.detection_metrics['total_predictions'] += len(predictions['boxes'])
            self.detection_metrics['total_ground_truth'] += len(gt_boxes)
            
            # Guardar visualización si se solicita
            if save_visualizations:
                self.save_visualization(image_path, predictions, gt_boxes, gt_labels, image_file)
        
        # Calcular métricas finales
        final_metrics = self.calculate_final_metrics()
        
        # Guardar resultados
        self.save_results(results_data, final_metrics)
        
        # Crear gráficos
        self.create_plots(results_data, final_metrics)
        
        print(f"✅ Evaluación completada. Resultados guardados en: {self.output_dir}")
        
        return final_metrics, results_data
    
    def load_ground_truth(self, image_file, labels_dir):
        """Cargar ground truth desde archivo YOLO"""
        if labels_dir is None:
            return [], []
            
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(labels_dir, label_file)
        
        boxes = []
        labels = []
        
        if os.path.exists(label_path):
            # Obtener dimensiones de imagen para convertir coordenadas
            image_path = os.path.join(os.path.dirname(labels_dir), 'images', image_file)
            img = cv2.imread(image_path)
            if img is not None:
                h, w = img.shape[:2]
                
                with open(label_path, 'r') as f:
                    for line in f.readlines():
                        parts = line.strip().split()
                        if len(parts) >= 5:
                            class_id = int(parts[0])
                            x_center = float(parts[1]) * w
                            y_center = float(parts[2]) * h
                            width = float(parts[3]) * w
                            height = float(parts[4]) * h
                            
                            # Convertir a formato xyxy
                            x1 = x_center - width / 2
                            y1 = y_center - height / 2
                            x2 = x_center + width / 2
                            y2 = y_center + height / 2
                            
                            boxes.append([x1, y1, x2, y2])
                            labels.append(class_id + 1)  # Convertir a formato Fast-RCNN
        
        return boxes, labels
    
    def calculate_image_metrics(self, predictions, gt_boxes, gt_labels, iou_threshold):
        """Calcula métricas para una imagen individual"""
        tp = 0
        fp = 0
        fn = 0
        
        if not predictions['boxes'] and not gt_boxes:
            # Imagen sin detecciones ni ground truth
            return {'tp': 0, 'fp': 0, 'fn': 0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}
        
        if not gt_boxes:
            # No hay ground truth, todas las predicciones son FP
            return {'tp': 0, 'fp': len(predictions['boxes']), 'fn': 0, 
                   'precision': 0.0, 'recall': 1.0, 'f1': 0.0}
        
        if not predictions['boxes']:
            # No hay predicciones, todos los GT son FN
            return {'tp': 0, 'fp': 0, 'fn': len(gt_boxes), 
                   'precision': 1.0, 'recall': 0.0, 'f1': 0.0}
        
        # Marcar ground truth como detectados
        gt_detected = [False] * len(gt_boxes)
        
        # Para cada predicción, buscar el mejor match en GT
        for pred_box, pred_label in zip(predictions['boxes'], predictions['labels']):
            best_iou = 0
            best_gt_idx = -1
            
            for gt_idx, (gt_box, gt_label) in enumerate(zip(gt_boxes, gt_labels)):
                if pred_label == gt_label and not gt_detected[gt_idx]:
                    iou = self.ensemble.calculate_iou(pred_box, gt_box)
                    if iou > best_iou:
                        best_iou = iou
                        best_gt_idx = gt_idx
            
            if best_iou >= iou_threshold and best_gt_idx >= 0:
                tp += 1
                gt_detected[best_gt_idx] = True
            else:
                fp += 1
        
        # Contar FN (GT no detectados)
        fn = sum(1 for detected in gt_detected if not detected)
        
        # Calcular métricas
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
        
        return {'tp': tp, 'fp': fp, 'fn': fn, 'precision': precision, 'recall': recall, 'f1': f1}
    
    def save_visualization(self, image_path, predictions, gt_boxes, gt_labels, image_name):
        """Guarda imagen con boxes de predicción y ground truth"""
        image = cv2.imread(image_path)
        if image is None:
            return
        
        # Copiar imagen para visualización
        vis_image = image.copy()
        
        # Dibujar ground truth en verde
        for gt_box, gt_label in zip(gt_boxes, gt_labels):
            x1, y1, x2, y2 = map(int, gt_box)
            cv2.rectangle(vis_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(vis_image, f'GT:{gt_label}', (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        # Dibujar predicciones en rojo
        for pred_box, pred_score, pred_label in zip(
            predictions['boxes'], predictions['scores'], predictions['labels']
        ):
            x1, y1, x2, y2 = map(int, pred_box)
            cv2.rectangle(vis_image, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(vis_image, f'P:{pred_label}({pred_score:.2f})', (x1, y2+15), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        
        # Agregar información en la imagen
        info_text = f"GT: {len(gt_boxes)} | Pred: {len(predictions['boxes'])}"
        cv2.putText(vis_image, info_text, (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Guardar imagen
        output_path = os.path.join(self.output_dir, "visualizations", f"result_{image_name}")
        cv2.imwrite(output_path, vis_image)
    
    def calculate_final_metrics(self):
        """Calcula métricas finales del conjunto completo"""
        tp = self.detection_metrics['true_positives']
        fp = self.detection_metrics['false_positives']
        fn = self.detection_metrics['false_negatives']
        
        # Métricas de detección
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
        
        # Accuracy (para detección de objetos, se calcula diferente)
        total_correct = tp
        total_samples = tp + fp + fn
        accuracy = total_correct / total_samples if total_samples > 0 else 0.0
        
        final_metrics = {
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'accuracy': accuracy,
            'true_positives': tp,
            'false_positives': fp,
            'false_negatives': fn,
            'total_predictions': self.detection_metrics['total_predictions'],
            'total_ground_truth': self.detection_metrics['total_ground_truth']
        }
        
        return final_metrics
    
    def save_results(self, results_data, final_metrics):
        """Guarda resultados en archivos"""
        # Guardar CSV con resultados por imagen
        df = pd.DataFrame(results_data)
        csv_path = os.path.join(self.output_dir, "results", "detailed_results.csv")
        df.to_csv(csv_path, index=False)
        
        # Guardar métricas finales
        metrics_path = os.path.join(self.output_dir, "results", "final_metrics.txt")
        with open(metrics_path, 'w') as f:
            f.write("=== MÉTRICAS FINALES DEL ENSEMBLE ===\n\n")
            f.write(f"Precision: {final_metrics['precision']:.4f}\n")
            f.write(f"Recall: {final_metrics['recall']:.4f}\n")
            f.write(f"F1 Score: {final_metrics['f1_score']:.4f}\n")
            f.write(f"Accuracy: {final_metrics['accuracy']:.4f}\n\n")
            f.write("=== CONTEOS ===\n")
            f.write(f"True Positives: {final_metrics['true_positives']}\n")
            f.write(f"False Positives: {final_metrics['false_positives']}\n")
            f.write(f"False Negatives: {final_metrics['false_negatives']}\n")
            f.write(f"Total Predicciones: {final_metrics['total_predictions']}\n")
            f.write(f"Total Ground Truth: {final_metrics['total_ground_truth']}\n")
            f.write(f"\nFecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        print(f"📊 Resultados guardados en: {csv_path}")
        print(f"📊 Métricas finales guardadas en: {metrics_path}")
    
    def create_plots(self, results_data, final_metrics):
        """Crea gráficos de resultados"""
        df = pd.DataFrame(results_data)
        
        # Crear figura con subplots
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('Evaluación del Ensemble - Métricas del Conjunto de Test', fontsize=16)
        
        # 1. Distribución de métricas por imagen
        axes[0, 0].hist(df['precision'], bins=20, alpha=0.7, color='blue', label='Precision')
        axes[0, 0].hist(df['recall'], bins=20, alpha=0.7, color='green', label='Recall')
        axes[0, 0].hist(df['f1'], bins=20, alpha=0.7, color='red', label='F1')
        axes[0, 0].set_title('Distribución de Métricas por Imagen')
        axes[0, 0].set_xlabel('Valor de Métrica')
        axes[0, 0].set_ylabel('Frecuencia')
        axes[0, 0].legend()
        
        # 2. Número de predicciones vs ground truth
        axes[0, 1].scatter(df['ground_truth_count'], df['predictions_count'], alpha=0.6)
        axes[0, 1].plot([0, df['ground_truth_count'].max()], [0, df['ground_truth_count'].max()], 'r--')
        axes[0, 1].set_title('Predicciones vs Ground Truth')
        axes[0, 1].set_xlabel('Ground Truth Count')
        axes[0, 1].set_ylabel('Predictions Count')
        
        # 3. Métricas finales (barras)
        metrics_names = ['Precision', 'Recall', 'F1 Score', 'Accuracy']
        metrics_values = [
            final_metrics['precision'], 
            final_metrics['recall'], 
            final_metrics['f1_score'], 
            final_metrics['accuracy']
        ]
        bars = axes[0, 2].bar(metrics_names, metrics_values, color=['blue', 'green', 'red', 'orange'])
        axes[0, 2].set_title('Métricas Finales del Ensemble')
        axes[0, 2].set_ylabel('Valor')
        axes[0, 2].set_ylim(0, 1)
        
        # Agregar valores en las barras
        for bar, value in zip(bars, metrics_values):
            height = bar.get_height()
            axes[0, 2].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                           f'{value:.3f}', ha='center', va='bottom')
        
        # 4. Matriz de confusión (simplificada)
        tp = final_metrics['true_positives']
        fp = final_metrics['false_positives']
        fn = final_metrics['false_negatives']
        tn = 0  # Para detección de objetos, TN es difícil de definir
        
        cm = np.array([[tp, fn], [fp, tn]])

        cell_labels = [
            f'Detectado como Célula\n(Predicción Positiva)',
            f'No Detectado como Célula\n(Predicción Negativa)'
        ]
        reality_labels = [
            f'Célula Real\n(Ground Truth Positivo)',
            f'Background Real\n(Ground Truth Negativo)'
        ]

        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                   xticklabels=cell_labels,
                   yticklabels=reality_labels, 
                   ax=axes[1, 0])
        
        axes[1, 0].set_title('Matriz de Confusión (Simplificada)')
        
        # 5. Distribución de TP, FP, FN por imagen
        metrics_counts = ['TP', 'FP', 'FN']
        tp_counts = df['true_positives'].sum()
        fp_counts = df['false_positives'].sum()
        fn_counts = df['false_negatives'].sum()
        
        axes[1, 1].bar(metrics_counts, [tp_counts, fp_counts, fn_counts], 
                      color=['green', 'red', 'orange'])
        axes[1, 1].set_title('Distribución Total de TP, FP, FN')
        axes[1, 1].set_ylabel('Cantidad')
        
        # 6. F1 Score por imagen (ordenado)
        df_sorted = df.sort_values('f1', ascending=False)
        axes[1, 2].plot(range(len(df_sorted)), df_sorted['f1'], 'b-', alpha=0.7)
        axes[1, 2].axhline(y=final_metrics['f1_score'], color='r', linestyle='--', 
                          label=f'F1 Promedio: {final_metrics["f1_score"]:.3f}')
        axes[1, 2].set_title('F1 Score por Imagen (Ordenado)')
        axes[1, 2].set_xlabel('Imagen (ordenadas por F1)')
        axes[1, 2].set_ylabel('F1 Score')
        axes[1, 2].legend()
        
        plt.tight_layout()
        
        # Guardar gráficos
        plots_path = os.path.join(self.output_dir, "metrics", "evaluation_plots.png")
        plt.savefig(plots_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"📈 Gráficos guardados en: {plots_path}")
        
        # Crear gráfico adicional: Precisión vs Recall por imagen
        plt.figure(figsize=(10, 8))
        plt.scatter(df['recall'], df['precision'], alpha=0.6, s=50)
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision vs Recall por Imagen')
        plt.grid(True, alpha=0.3)
        
        # Agregar punto para métricas promedio
        plt.scatter(final_metrics['recall'], final_metrics['precision'], 
                   color='red', s=100, marker='x', 
                   label=f'Promedio (P:{final_metrics["precision"]:.3f}, R:{final_metrics["recall"]:.3f})')
        plt.legend()
        
        pr_plot_path = os.path.join(self.output_dir, "metrics", "precision_recall_plot.png")
        plt.savefig(pr_plot_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"📈 Gráfico Precision-Recall guardado en: {pr_plot_path}")

In [ ]:
# Crear el ensemble
consensus_ensemble = ConsensusEnsemble(
    fast_rcnn_path="./Fast_RCNN_models/best_model.pth",
    yolo_path="./runs/detect/yolov11ny_new/weights/best.pt",
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

# Configurar parámetros conservadores
consensus_ensemble.consensus_iou_threshold = 0.5
consensus_ensemble.min_confidence = 0.7

# Crear evaluador
evaluator = ConsensusEnsembleEvaluator(
    consensus_ensemble=consensus_ensemble,
    output_dir="..\\03.Datasets\\Evaluacion_Empresa\\DEFAULT\\Ensemble"
)

# Evaluar conjunto de test completo
final_metrics, detailed_results = evaluator.evaluate_test_set(
    test_images_dir="../03.Datasets/YOLO_Datasets/test/images",
    test_labels_dir="../03.Datasets/YOLO_Datasets/test/labels",
    iou_threshold=0.5,
    save_visualizations=True
)

# Mostrar resultados finales
print("\n" + "="*50)
print("🎯 RESULTADOS FINALES DEL ENSEMBLE")
print("="*50)
print(f"Precision: {final_metrics['precision']:.4f}")
print(f"Recall: {final_metrics['recall']:.4f}")
print(f"F1 Score: {final_metrics['f1_score']:.4f}")
print(f"Accuracy: {final_metrics['accuracy']:.4f}")
print(f"\nTrue Positives: {final_metrics['true_positives']}")
print(f"False Positives: {final_metrics['false_positives']}")
print(f"False Negatives: {final_metrics['false_negatives']}")
print(f"\nTotal Predicciones: {final_metrics['total_predictions']}")
print(f"Total Ground Truth: {final_metrics['total_ground_truth']}")

🚀 Iniciando evaluación del conjunto de test...
📁 Directorio de imágenes: ../03.Datasets/YOLO_Datasets/test/images
📁 Directorio de salida: ..\03.Datasets\Evaluacion_Empresa\DEFAULT\Ensemble
📊 Total de imágenes a evaluar: 87
🔍 Procesando 1/87: 110.jpg

image 1/1 c:\Users\mini7\Desktop\Master\Materias\TFM\Laura\2025_MURIA_Aitor Garca Blanco\04.Codigo\..\03.Datasets\YOLO_Datasets\test\images\110.jpg: 1024x1280 1 cell, 97.3ms
Speed: 12.9ms preprocess, 97.3ms inference, 4.5ms postprocess per image at shape (1, 3, 1024, 1280)
Fast-RCNN: 9 detecciones
YOLO: 1 detecciones
Consenso: 0 detecciones
🔍 Procesando 2/87: 116.jpg

image 1/1 c:\Users\mini7\Desktop\Master\Materias\TFM\Laura\2025_MURIA_Aitor Garca Blanco\04.Codigo\..\03.Datasets\YOLO_Datasets\test\images\116.jpg: 1024x1280 28 cells, 17.0ms
Speed: 7.5ms preprocess, 17.0ms inference, 2.2ms postprocess per image at shape (1, 3, 1024, 1280)
Fast-RCNN: 35 detecciones
YOLO: 28 detecciones
Consenso: 25 detecciones
🔍 Procesando 3/87: 124.jpg

ima